# Merging the pairs

`bbmerge` package was used in this step

In [13]:
%%bash
input_list_file="sample_list.txt"
while IFS='' read -r line || [[ -n "$line" ]]; do
  sample_list="$sample_list $line"
done <"$input_list_file"
echo $sample_list

for read in $sample_list; do
    bbmerge-auto.sh in=$read'_R1.fastq' \
               in2=$read'_R2.fastq' \
               out='corrected_'$read'_R1.fastq' \
               out2='corrected_'$read'_R2.fastq' \
               ecco=t \
               mix=t

    bbmerge-auto.sh minoverlap=6 \
                    mininsert=20 \
                    mininsert0=20 \
                    in='corrected_'$read'_R1.fastq' \
                    in2='corrected_'$read'_R2.fastq' \
                    out="merged_"$read".fastq" \
                    outu="unmerged_"$read"_R1.fastq" \
                    outu2="unmerged_"$read"_R2.fastq" 
done

LM1 LM4 LM5


java -ea -Xmx24675m -Xms24675m -cp /usr/local/bin/bbmap/current/ jgi.BBMerge in=LM1_R1.fastq in2=LM1_R2.fastq out=corrected_LM1_R1.fastq out2=corrected_LM1_R2.fastq ecco=t mix=t
Executing jgi.BBMerge [in=LM1_R1.fastq, in2=LM1_R2.fastq, out=corrected_LM1_R1.fastq, out2=corrected_LM1_R2.fastq, ecco=t, mix=t]
Version 38.86

Writing mergable reads unmerged in two files.
Started output threads.
Total time: 93.728 seconds.

Pairs:               	34375177
Joined:              	16325022   	47.491%
Ambiguous:           	18050147   	52.509%
No Solution:         	8       	0.000%
Too Short:           	0       	0.000%
Errors Corrected:    	552636

Avg Insert:          	59.3
Standard Deviation:  	9.5
Mode:                	70

Insert range:        	35 - 74
90th percentile:     	71
75th percentile:     	67
50th percentile:     	61
25th percentile:     	53
10th percentile:     	45
java -ea -Xmx24520m -Xms24520m -cp /usr/local/bin/bbmap/current/ jgi.BBMerge minoverlap=6 mininsert=20 mininsert0=20 in=cor

# Move UMI to read name and Process with merged reads only

`cutadapt`, as well as `move_umi.py` from https://github.com/yz201906/RibOxi-seq was used in this step

In [14]:
%%bash
input_list_file="sample_list.txt"
while IFS='' read -r line || [[ -n "$line" ]]; do
  sample_list="$sample_list $line"
done <"$input_list_file"
echo $sample_list

for read in $sample_list; do
    cat 'merged_'$read'.fastq' "unmerged_"$read"_R1.fastq" > $read'.fastq'


    cutadapt -j 12 \
        --discard-untrimmed \
        --minimum-length 20 \
        -a CTGTAGGCACCATCAATGAC \
        -e 0.1 \
        -o 'dt_'$read'.fastq' $read'.fastq'


    move_umi.py -r 'dt_'$read'.fastq' \
                -l 4 \
                -o 'umiRemoved_'"$read" \
                -i ATCACG \
                -a ATCACGCTGTAGGCACCATCAATGAC \
                -m 'pipeline'

    cutadapt -j 12 \
        --discard-untrimmed \
        --minimum-length 20 \
        -a ATCACG \
        -e 0.01 \
        -o 'trimmed_'$read'.fastq' 'umiRemoved_'$read'_R1.fastq'

done

LM1 LM4 LM5
This is cutadapt 2.10 with Python 3.8.5
Command line parameters: -j 12 --discard-untrimmed --minimum-length 20 -a CTGTAGGCACCATCAATGAC -e 0.1 -o dt_LM1.fastq LM1.fastq
Processing reads on 12 cores in single-end mode ...
Finished in 31.64 s (1 us/read; 65.19 M reads/minute).

=== Summary ===

Total reads processed:              34,375,177
Reads with adapters:                17,854,174 (51.9%)
Reads written (passing filters):    16,214,824 (47.2%)

Total basepairs processed: 1,714,674,384 bp
Total written (filtered):    648,257,920 bp (37.8%)

=== Adapter 1 ===

Sequence: CTGTAGGCACCATCAATGAC; Type: regular 3'; Length: 20; Trimmed: 17854174 times

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20 bp: 2

Bases preceding removed adapters:
  A: 2.3%
  C: 0.5%
  G: 96.2%
  T: 1.0%
  none/other: 0.0%
    The adapter is preceded by "G" extremely often.
    The provided adapter sequence could be incomplete at its 3' end.

Overview of removed sequences
length	count	expect	max.err	err

# Process with just Read1 only

# Alignment

`STAR` was used to perform sequence alignment against curated *Leishmania* rRNA sequences.   

In [15]:
%%bash
input_list_file="sample_list.txt"
while IFS='' read -r line || [[ -n "$line" ]]; do
  sample_list="$sample_list $line"
done <"$input_list_file"
for sample in $sample_list; do
    mkdir -p $sample
    STAR --runThreadN 10 \
      --outMultimapperOrder Random \
      --outFilterMismatchNoverLmax 0.1 \
      --alignEndsType EndToEnd \
      --limitOutSJcollapsed 20000000 \
      --limitIObufferSize 550000000 \
      --limitBAMsortRAM 2960722171 \
      --genomeDir $GENOMES"/LM/" \
      --readFilesIn 'trimmed_'"$sample"'.fastq' \
      --outFileNamePrefix ./"$sample"/ \
      --genomeLoad LoadAndKeep \
      --outSAMtype BAM SortedByCoordinate
#       --outFilterScoreMinOverLread 0.8 \
#       --outFilterMatchNminOverLread 0.8 \
done

STAR --genomeDir $GENOMES"/LM/" --genomeLoad Remove

May 11 23:04:07 ..... started STAR run
May 11 23:04:07 ..... loading genome
May 11 23:04:07 ..... started mapping
May 11 23:04:34 ..... finished mapping
May 11 23:04:35 ..... started sorting BAM
May 11 23:04:52 ..... finished successfully
May 11 23:04:53 ..... started STAR run
May 11 23:04:53 ..... loading genome
May 11 23:04:53 ..... started mapping
May 11 23:05:16 ..... finished mapping
May 11 23:05:16 ..... started sorting BAM
May 11 23:05:33 ..... finished successfully
May 11 23:05:34 ..... started STAR run
May 11 23:05:34 ..... loading genome
May 11 23:05:34 ..... started mapping
May 11 23:05:51 ..... finished mapping
May 11 23:05:52 ..... started sorting BAM
May 11 23:06:02 ..... finished successfully
May 11 23:06:06 ..... started STAR run
May 11 23:06:06 ..... loading genome
May 11 23:06:06 ..... started mapping
May 11 23:06:06 ..... finished mapping
May 11 23:06:06 ..... finished successfully


# Post alignment processing

`samtools` was used to index the alignment output.  
`umi_tools` was used to remove PCR duplicates.  
`bedtools` was used to generate human readable bed files for downstream analysis.  

In [16]:
%%bash
# Read in sample from out list and store as a variable:
input_list_file="sample_list.txt"
while IFS='' read -r line || [[ -n "$line" ]]; do
  sample_list="$sample_list $line"
done <"$input_list_file"
for sample in $sample_list; do
  cd "$sample"
  samtools index "Aligned.sortedByCoord.out.bam"
  umi_tools dedup -I "Aligned.sortedByCoord.out.bam" -S $sample'_dedup.bam'
  bedtools bamtobed -i $sample'_dedup.bam' >'../bed_files/SE_aligned/'$sample'.bed'
  cd ../
done

# UMI-tools version: 1.0.1
# output generated by dedup -I Aligned.sortedByCoord.out.bam -S LM1_dedup.bam
# job started at Tue May 11 23:07:43 2021 on yzylhome -- 8135461f-fd54-494c-8f89-31ea84f213bb
# pid: 8184, system: Linux 5.4.0-73-generic #82-Ubuntu SMP Wed Apr 14 17:39:42 UTC 2021 x86_64
# assigned_tag                            : None
# cell_tag                                : None
# cell_tag_delim                          : None
# cell_tag_split                          : -
# chimeric_pairs                          : use
# chrom                                   : None
# compresslevel                           : 6
# detection_method                        : None
# gene_tag                                : None
# gene_transcript_map                     : None
# get_umi_method                          : read_id
# ignore_umi                              : False
# in_sam                                  : False
# log2stderr                              : False
# loglevel           